In [1]:
import os#
#os.chdir('c:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\STK_SEARCH\\')
os.chdir("/rds/general/user/ma11115/home/STK_Search/STK_search")
from pathlib import Path#
import pandas as pd
import torch
import numpy as np
import importlib
from stk_search.utils import database_utils
from stk_search.utils import config_utils
import datetime
from stk_search.utils import update_datasets
import importlib
importlib.reload(config_utils)
os.makedirs('data/output/search_experiment/search_exp_database', exist_ok=True)

In [2]:
## load data from the database
df_precursors_path ="data/output/Prescursor_data/precursor_data_new_clustering.pkl"

df_total_new , df_precursors= database_utils.load_data_database(df_precursors_path,num_fragm=6)

/rds/general/user/ma11115/home/STK_Search/STK_search/src/stk_search/utils/database_utils.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f"InChIKey_{i}"] = df_test["BB"].apply(


In [9]:
#Helper function to split the dataset


def do_random_split(config,hide_best_molecules=True,df_total_new=df_total_new):
    df = df_total_new.copy()
    def topk_split(topkmol, df,num_molecules_dataset=5000,target_name='target',hide_best_molecules=hide_best_molecules):
        min_target_out_of_database = -np.sort(-df[target_name].values)[topkmol]
        df_test = df[df[target_name]>=min_target_out_of_database]
        if hide_best_molecules:
            df_train = df[df[target_name]<min_target_out_of_database]
        else:
            df_train = df.copy()
            print(df_train.shape[0])
        # add random element from df to the test set
        #df_test_random = df.sample(n=1000, random_state=1)
        #df_test = pd.concat([df_test, df_test_random])
        df_test.reset_index(drop=True, inplace=True)
        list_col = [f'InChIKey_{x}' for x in range(6)]
        list_col.append(target_name)
        list_col.append('InChIKey')
        df_train = df_train[list_col]
        df_train = df_train[~df_train['InChIKey'].isin(df_test['InChIKey'])]
        if num_molecules_dataset>df_train.shape[0]:
            print("Warning: the number of molecules in the dataset is smaller than the number of molecules requested")
            num_molecules_dataset = df_train.shape[0]
        df_train = df_train.sample(num_molecules_dataset)
        df_val = df_train.sample(frac=0.1, random_state=1)
        df_train = df_train[~df_train['InChIKey'].isin(df_val['InChIKey'])]
        df_train.reset_index(drop=True, inplace=True)
        df_val.reset_index(drop=True, inplace=True)
        return df_train, df_val, df_test
    topKmol = int(df.shape[0]*0.01)
    df_train, df_val, df_test = topk_split(topKmol, df,config["num_molecules"],config["target_name"],hide_best_molecules=hide_best_molecules)
    config_dir= config["running_dir"]
    df_train.to_csv(config_dir + "df_train.csv", index=False)
    df_val.to_csv(config_dir + "df_val.csv", index=False)
    df_test.to_csv(config_dir + "df_test.csv", index=False)
    return df_train, df_val, df_test



In [10]:

importlib.reload(update_datasets)
importlib.reload(config_utils)
date = datetime.datetime.now().strftime("%y%m%d")
#df_total_path = df_total_path_new
config_dir_old =  "data/representation_learning/6-frag/target/240517/SchNet/Full_data/"
config_dir_to_save = f"data/representation_learning/6-frag/target/{date}//SchNet/Full_data/"

os.makedirs(config_dir_to_save, exist_ok=True)

In [15]:
os.chdir("/rds/general/user/ma11115/home/STK_Search/STK_search")

config = config_utils.load_config(config_dir_old)
config["num_molecules"] = df_total_new.shape[0]
config["running_dir"] = "/rds/general/user/ma11115/home/STK_Search/STK_search/"+config_dir_to_save
config_dir = config["running_dir"]
if config["split_type"] == "rand":
    df_train, df_val, df_test = do_random_split(config,hide_best_molecules=False)
for _case in ['test','train','val']:
    config[f"dataset_path_{_case}"] = ""
config["device"] = "cuda" if torch.cuda.is_available() else "cpu"
config['name'] = f"6-frag_target_{date}__SchNet_full_data_nummol{config['num_molecules']}"
config_utils.save_config(config,config_dir_to_save)

58320


In [17]:
#retrain 
os.chdir("/rds/general/user/ma11115/home/STK_Search/STK_search")

%run src/dev_scripts/run_representation_learning_polymer.py --config_dir {config_dir}

config loaded from /rds/general/user/ma11115/home/STK_Search/STK_search/data/representation_learning/6-frag/target/240520//SchNet/Full_data/
dataset not found


Pandas Apply:   0%|          | 0/51962 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/51962 [00:00<?, ?it/s]

dataset not found


Pandas Apply:   0%|          | 0/5774 [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
lr,████████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▂▂▁▁
train_loss,▅█▅▄▃▆▃▃▄▄▃▃▂▂▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂
train_loss_repr,▄▄▄▆▄▃█▄▅▅▆▅▇▇▅▅▄▅▄▄▅▄▇▄▄▅▅▃▂▃▃▁▃▃▂▃▁▂▂▁
train_loss_target,▅█▅▄▃▆▃▃▄▄▃▃▂▂▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▅█▂▃▂▅▃▂▁▄▃▃▄▁▄▃▇▂▁▆▃▃▄▄▄▆▂▁▃
val_loss_repr,▆▇▆▅▆█▇▆▇█▇▇▆▅▅▅▅▄▄▃▃▃▃▃▂▂▂▁▁
val_loss_target,▅█▂▃▂▅▃▂▁▄▃▃▄▂▄▃▇▂▁▇▃▄▄▄▄▇▃▂▄
epoch,28
lr,0.00048


Total time taken for model training: 3288.2162573337555 seconds
config loaded from /rds/general/user/ma11115/home/STK_Search/STK_search/data/representation_learning/6-frag/target/240520//SchNet/Full_data/
/rds/general/user/ma11115/home/STK_Search/STK_search/data/representation_learning/6-frag/target/240520//SchNet/Full_data/epoch=8-val_loss=0.12-other_metric=0.00.ckpt
Model loaded:  SchNet
Model loaded
Model evaluation done
MAE: 0.43, MSE: 0.37, R2: 0.71
MAE: 0.43, MSE: 0.36, R2: 0.71
MAE: 0.48, MSE: 0.29, R2: -85.48


In [9]:
config["running_dir"]

'data/representation_learning/6-frag/target/240517//SchNet/Full_data/'